In [1]:
import autograd
from autograd import numpy as np

In [2]:
from collections import OrderedDict
import paragami

In [3]:
import time

a = np.full(100000, float('nan'))
b = np.random.random(a.shape)

def set_inplace(b, a):
    a[:] = b + 2
    return a

def set_notinplace(b):
    return b + 2

set_inplace(b, a)

t1 = time.time()
for iter in range(1000):
    set_inplace(b, a)
print('Inplace: ', time.time() - t1)

t1 = time.time()
for iter in range(1000):
    a = set_notinplace(b)
print('Not in place: ', time.time() - t1)

# WTF?  Try %memit and id

Inplace:  0.160536527633667
Not in place:  0.06052041053771973


In [4]:


def check_dict_equal(dict1, dict2):
    assert dict1.keys() == dict2.keys()
    for key  in dict1:
        assert_array_almost_equal(dict1[key], dict2[key])

dict_pattern = paragami.PatternDict()
dict_pattern['a'] = paragami.NumericArrayPattern((2, 3, 4), lb=-1, ub=2)
dict_pattern['b'] = paragami.NumericArrayPattern((5, ), lb=-1, ub=10)
dict_pattern['c'] = paragami.NumericArrayPattern((5, 2), lb=-1, ub=10)

dict_val = dict_pattern.random()
print(dict_val)
dict_val2 = dict(dict_val)
print(dict_val2)
flat_val1 = dict_pattern.flatten(dict_val, free=True)
flat_val2 = dict_pattern.flatten(dict_val2, free=True)

np.linalg.norm(flat_val1 - flat_val2)

OrderedDict([('a', array([[[0.57977793, 0.83946051, 0.86789968, 1.04968666],
        [1.13662161, 1.16181468, 0.63690338, 0.67688828],
        [0.92360404, 0.74506489, 1.18423022, 0.78782293]],

       [[0.64000621, 1.01738151, 0.85246576, 0.61130373],
        [0.97033267, 1.08805055, 0.58569253, 1.12878684],
        [0.79927959, 0.74373808, 0.9650658 , 0.77305853]]])), ('b', array([5.11140704, 6.09051081, 5.2625492 , 6.21552663, 5.99366008])), ('c', array([[6.83630556, 6.59108777],
       [4.58160455, 7.00839067],
       [6.33830807, 6.84705049],
       [5.12426865, 6.53362792],
       [6.86742689, 5.22218362]]))])
{'a': array([[[0.57977793, 0.83946051, 0.86789968, 1.04968666],
        [1.13662161, 1.16181468, 0.63690338, 0.67688828],
        [0.92360404, 0.74506489, 1.18423022, 0.78782293]],

       [[0.64000621, 1.01738151, 0.85246576, 0.61130373],
        [0.97033267, 1.08805055, 0.58569253, 1.12878684],
        [0.79927959, 0.74373808, 0.9650658 , 0.77305853]]]), 'b': array([5.111

0.0

In [5]:
param_dict = dict_pattern.random()

def test_function(param_dict):
    a = param_dict['a']
    b = param_dict['b']
    return np.mean(a ** 2) + np.mean(b ** 2)

test_function(param_dict)

40.28487845683647

In [6]:
class PatternedFunction:
    def __init__(self, original_fun, pattern, free, argnum=0):
        self._fun = original_fun
        self._argnum = argnum
        self.free = free
        self._pattern = pattern
        
    def __str__(self):
        return('Function: {}\nargnum: {}\nfree: {}\npattern: {}'.format(
            self._fun, self._argnum, self.free, self._pattern))
    
    def __call__(self, *args, **kwargs):
        flat_val = args[self._argnum]
        folded_val = self._pattern.fold(flat_val, free=self.free)
        new_args = args[0:self._argnum] + (folded_val, ) + args[self._argnum + 1:-1]
        return self._fun(*new_args, **kwargs)

    
patterned_test_function = PatternedFunction(test_function, dict_pattern, True)
print(patterned_test_function)

flat_param_dict = dict_pattern.flatten(param_dict, free=True)
assert(np.abs(patterned_test_function(flat_param_dict) - test_function(param_dict)) < 1e-12)

patterned_test_function_grad = autograd.grad(patterned_test_function)
patterned_test_function_grad(flat_param_dict)

patterned_test_function_hessian = autograd.hessian(patterned_test_function)
patterned_test_function_hessian(flat_param_dict)

Function: <function test_function at 0x7f8e41a4bb70>
argnum: 0
free: True
pattern: OrderedDict:
	[a] = Array (2, 3, 4) (lb=-1, ub=2)
	[b] = Array (5,) (lb=-1, ub=10)
	[c] = Array (5, 2) (lb=-1, ub=10)
offset:  0
pattern_flat_length:  24
flat_val:  [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan]
slice:  [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan]
folded val:  OrderedDict([('a', array([[[0.79175053, 0.87332182, 0.91132766, 0.53796117],
        [1.04279574, 0.93281648, 0.89588204, 0.66334994],
        [0.58888268, 1.03945712, 0.91120697, 0.9926655 ]],

       [[0.80714611, 1.10794515, 0.80566549, 0.79093677],
        [0.77210089, 0.69745899, 0.76617192, 1.01039665],
        [1.06723176, 0.90159182, 0.51963886, 0.97887435]]])), ('b', array([6.95467331, 4.64650896, 6.41240482, 6.89642728, 6.24836232])), ('c', array([[6.5696906

array([[0.03413486, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.02850467, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.025646  , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [7]:
class FlattenedFunction:
    def __init__(self, original_fun, patterns, free, argnums=None):
        self._fun = original_fun
        self._patterns = np.atleast_1d(patterns)
        if argnums is None:
            argnums = np.arange(0, len(self._patterns))
        if len(self._patterns.shape) != 1:
            raise ValueError('patterns must be a 1d vector.')
        self._argnums = np.atleast_1d(argnums)
        self._argnum_sort = np.argsort(self._argnums)
        self.free = np.broadcast_to(free, self._patterns.shape)

        self._validate_args()
        
    def _validate_args(self):
        if len(self._argnums.shape) != 1:
            raise ValueError('argnums must be a 1d vector.')
        if len(self._argnums) != len(self._patterns):
            raise ValueError('argnums must be the same length as patterns.')
        if len(self.free.shape) != 1:
            raise ValueError('free must be a single boolean or a 1d vector of booleans.')
        if len(self.free) != len(self._patterns):
            raise ValueError('free must broadcast to the same shape as patterns.')
        
    def __str__(self):
        return('Function: {}\nargnums: {}\nfree: {}\npatterns: {}'.format(
            self._fun, self._argnums, self.free, self._patterns))
    
    def __call__(self, *args, **kwargs):
        # Loop through the arguments from beginning to end, replacing parameters
        # with their flattened values.
        new_args = ()
        last_argnum = 0
        for i in self._argnum_sort:
            argnum = self._argnums[i]
            folded_val = self._patterns[i].fold(args[argnum], free=self.free[i])
            new_args += args[last_argnum:argnum] + (folded_val, )
            last_argnum = argnum + 1
            
        return self._fun(*new_args, **kwargs)


def test_function(x, a, y, b):
    return x ** 2 + y ** 2 + np.mean(a ** 2) + np.mean(b ** 2)


x = 2
y = 3
a = param_dict['a']
b = param_dict['b']
a_flat = dict_pattern['a'].flatten(a, True)
b_flat = dict_pattern['b'].flatten(b, True)


patterns = [dict_pattern['a'], dict_pattern['b']]
flat_test_function = FlattenedFunction(
    test_function, patterns, free=[True, True], argnums=[1, 3])

assert(np.linalg.norm(
    test_function(x, a, y, b) - flat_test_function(x, a_flat, y, b_flat) < 1e-9))

print(flat_test_function)




def test_function(param_dict):
    a = param_dict['a']
    b = param_dict['b']
    return np.mean(a ** 2) + np.mean(b ** 2)

flat_test_function = FlattenedFunction(
    test_function, dict_pattern, free=True)
param_dict_flat = dict_pattern.flatten(param_dict, free=True)
assert(np.linalg.norm(
    test_function(param_dict) - flat_test_function(param_dict_flat) < 1e-9))



Function: <function test_function at 0x7f8e41a66ae8>
argnums: [1 3]
free: [ True  True]
patterns: [<paragami.numeric_array_patterns.NumericArrayPattern object at 0x7f8e41a632b0>
offset:  0
pattern_flat_length:  24
flat_val:  [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan]
slice:  [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan]
folded val:  OrderedDict([('a', array([[[0.79175053, 0.87332182, 0.91132766, 0.53796117],
        [1.04279574, 0.93281648, 0.89588204, 0.66334994],
        [0.58888268, 1.03945712, 0.91120697, 0.9926655 ]],

       [[0.80714611, 1.10794515, 0.80566549, 0.79093677],
        [0.77210089, 0.69745899, 0.76617192, 1.01039665],
        [1.06723176, 0.90159182, 0.51963886, 0.97887435]]])), ('b', array([6.95467331, 4.64650896, 6.41240482, 6.89642728, 6.24836232])), ('c', array([[6.56969061, 6.15444495],
       

In [8]:
import datetime
np.array([datetime.datetime.now()])

array([array(datetime.datetime(2018, 10, 25, 15, 54, 29, 108348), dtype=object)],
      dtype=object)

In [9]:
a = np.random.random((2, 2))
print('a: ', a)

shape = (3, 4)
foo = np.empty(shape, dtype='object')
for i in range(3):
    for j in range(4):
        foo[i, j] = a
print(foo.shape)
print(foo[0, 1])

a:  [[0.14625825 0.28894702]
 [0.05086982 0.77470354]]
(3, 4)
[[0.14625825 0.28894702]
 [0.05086982 0.77470354]]


In [10]:
a = np.random.random((2, 2))
print(a)
shape = (3, 4)
#foo = np.reshape(np.array([a for i in range(np.prod(shape))]), shape + a.shape)
foo = np.array([a for i in range(np.prod(shape))])
print(foo.shape)
print(foo.dtype)
print('With shape broadcasting:\n', foo[0, 0])

print('Of a dict:')
a = dict(a=5)
shape = (3, 4)
foo = np.reshape(np.array([a for i in range(np.prod(shape))], dtype='object'), shape)
print(foo.shape)
print(foo[0, 0])


print('\nOf an object array:')
a = np.array(np.random.random((2, 2)), dtype='object')
print(a)
shape = (3, 4)
foo = np.reshape(np.array([a for i in range(np.prod(shape))]), shape + a.shape)
print(foo.shape)
print(foo.dtype)


[[0.78919905 0.66628686]
 [0.5464367  0.6067787 ]]
(12, 2, 2)
float64
With shape broadcasting:
 [0.78919905 0.66628686]
Of a dict:
(3, 4)
{'a': 5}

Of an object array:
[[0.1717951676260866 0.978156907664484]
 [0.07656300683740247 0.0911246880093024]]
(3, 4, 2, 2)
object


In [11]:
def is_np(a):
    return(type(a) is np.ndarray)

print(is_np(np.random.random((2, 2))))

a = np.array(np.random.random((2, 2)), dtype='object')
print(is_np(a))

print(is_np(dict(a=4)))


True
True
False


In [12]:
# Array
from paragami.base_patterns import PatternArray

pattern_array = PatternArray(
    (2, 3), paragami.NumericArrayPattern(shape=(2, ), lb=-1, ub=10.0))

val = pattern_array.random()
pattern_array.flatten(val, True)
pattern_array.flatten(val, False)



array([5.48438571, 6.20300891, 4.57577737, 5.71077412, 6.55340708,
       6.0590386 , 5.04452103, 5.44314232, 6.77435478, 4.74702353,
       6.75193052, 5.46834599])

In [47]:
import numpy as onp

(2, 3, ) + ()
foo = OrderedDict(a=5)
bar = onp.array([foo for i in range(3)])
print(bar[0])
# OrderedDict([('a', 5)])

print(type(bar[0]))
# <class 'numpy.ndarray'>

print(bar.dtype)
# object

print(bar[0].shape)
# ()

print(bar[0]['a'])
# IndexError

print(bar[0][0])
# IndexError: too many indices for array


OrderedDict([('a', 5)])
<class 'collections.OrderedDict'>
object


AttributeError: 'collections.OrderedDict' object has no attribute 'shape'

In [27]:

dict_pattern = paragami.PatternDict()
dict_pattern['a'] = paragami.NumericArrayPattern(shape=(2, ), lb=-2, ub=10.0)
pattern_array = PatternArray((2, 3), dict_pattern)

val = pattern_array.random()
print(val.shape)
print(type(val[0, 0]))
print(val[0, 0])

# __array_ranges = [ range(0, t) for t in (2, 3) ]

# print(__array_ranges)
# pattern_array.flatten(val, True)
#pattern_array.flatten(val, False)


(2, 3)
<class 'numpy.ndarray'>
OrderedDict([('a', array([6.1133247 , 6.01434147]))])
